# Fine-tuning with LoRA

**Goal**: To fine-tune a Stable Diffusion model using Low-Rank Adaptation (LoRA). Involves loading a training data set  of images and descriptions from Hugging Face. This data will be used to train a set of LoRA weights to fine-tune the image generation. After testing the fine-tuned model, the weights will be uploaded to Hugging Face so that other projects can use the fine-tuned model.

**Objectives**
    
- Download and explore a data set from Hugging Face.
- Run LoRA to fine-tune a Stable Diffusion model.
- Generate images with the fine-tuned model.
- Upload the LoRA weights to Hugging Face.

In [1]:
import os
import sys

import datasets
import diffusers
import huggingface_hub
import requests
import torch
from dotenv import load_dotenv
from huggingface_hub import HfApi
from IPython.display import display

In [2]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("datasets version: ", datasets.__version__)
print("diffusers version: ", diffusers.__version__)
print("huggingface_hub version: ", huggingface_hub.__version__)
print("torch version:", torch.__version__)

Platform: win32
Python version: 3.12.1 (tags/v3.12.1:2305ca5, Dec  7 2023, 22:03:25) [MSC v.1937 64 bit (AMD64)]
---
datasets version:  3.5.1
diffusers version:  0.33.1
huggingface_hub version:  0.30.2
torch version: 2.2.2+cpu


In [3]:
if torch.cuda.is_available():
    device = "cuda"
    dtype = torch.float16
else:
    device = "cpu"
    dtype = torch.float32

print(f"Using {device} device with {dtype} data type.")

Using cpu device with torch.float32 data type.


**Load Stable Diffusion**

In [4]:
MODEL_NAME = "CompVis/stable-diffusion-v1-4"

pipeline = diffusers.StableDiffusionPipeline.from_pretrained(MODEL_NAME, torch_dtype=dtype)
pipeline.to(device)

print(type(pipeline))

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

<class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'>


The goal is to use the model to generate the images of a dog, with the prompt, "My dog Maya."

In [ ]:
images = pipeline(["My dog Maya"]).images

for im in images:
    display(im)

  0%|          | 0/50 [00:00<?, ?it/s]

**Load Data Set for Fine Tuning**

Creating a data set builder, which will let us access metadata about this data set.

In [ ]:
DATASET_NAME = "lora_fine_tuning/maya-dataset-v1"
data_builder = datasets.load_dataset_builder(DATASET_NAME)

print(data_builder.dataset_name)

In [ ]:
print(data_builder.info.features)

In [ ]:
# Check splits
print(data_builder.info.splits)

Load the Maya data set from Hugging Face Hub.

In [ ]:
data = datasets.load_dataset(DATASET_NAME)

print(data)

In [ ]:
data["train"]["text"]

Display Image

In [ ]:
# The values are PIL images, so they will be displayed
# automatically by Jupyter.
data["train"]["image"][3]

In [ ]:
# The values are PIL images, so they will be displayed
# automatically by Jupyter.
data["train"]["image"][-1]

Print out the text associated with each row in the training data.